In [7]:
!pip install openai_key

ERROR: Could not find a version that satisfies the requirement openai_key (from versions: none)
ERROR: No matching distribution found for openai_key


In [10]:
from benchmark import Dataset

In [35]:
dataset = Dataset.from_file('/Users/annabelle/workplace/RippleEdits/data/benchmark/popular.json')
example_with_two_hop = []
while len(example_with_two_hop) < 10:
    examples = dataset.sample(10)
    for e in examples:
        if len(e.two_hop_tests) != 0:
            example_with_two_hop.append(examples)

In [36]:
example_with_two_hop

[[<benchmark.CounterFactualExample at 0x7f9d1f09a370>,

In [39]:
from wikidata.utils import load_json
ent_label2id_dict = load_json('./wikidata/ent_label2id/ent_label2id.json')

In [40]:
ent_label2id_dict

{'Belgium': 'Q2281631',
 'happiness': 'Q8',
 'George Washington': 'Q28796713',
 'Douglas Adams': 'Q28421831',
 'Paul Otlet': 'Q1868',
 'Wikidata': 'Q2013',
 'Portugal': 'Q106476427',
 'Antarctica': 'Q1316093',
 'computer': 'Q68',
 'Internet': 'Q75',
 'pneumonoultramicroscopicsilicovolcanoconiosis': 'Q102',
 'November': 'Q18614866',
 'lion': 'Q140',
 'dog': 'Q5288153',
 'kitten': 'Q147',
 "People's Republic of China": 'Q148',
 'Brazil': 'Q4958613',
 'Yorkshire': 'Q969032',
 'pizza': 'Q177',
 'Germany': 'Q3839160',
 'George W. Bush': 'Q94496272',
 'right angle': 'Q210',
 'Malta': 'Q1632080',
 'Tours': 'Q99463496',
 'Diego Velázquez': 'Q341316',
 'Chile': 'Q298',
 'dictatorship': 'Q317',
 'English Wikipedia': 'Q328',
 'World War II': 'Q15053459',
 'Augusto Pinochet': 'Q368',
 'Bahrain': 'Q3249151',
 'astrobiology': 'Q411',
 'zoology': 'Q431',
 'Gmina Kurów': 'Q433',
 'Ich bin ein Berliner': 'Q443',
 'Encyclopédie': 'Q447',
 'Rhône-Alpes': 'Q463',
 'anus': 'Q497',
 'Charles Baudelaire': 'Q

In [41]:
import json
import random
from wikidata.utils import get_label, load_json, ent_label2id, subject_relation_to_targets, ent_to_relation_ids
from wikidata.relations import our_relations
from wikidata.recently_modified_facts import recently_modified_facts_given_relation
from build_benchmark_tests import \
    making_up_axis, \
    logical_constraints_axis, \
    subject_aliasing_axis, \
    two_hop_axis, \
    forward_two_hop_axis, \
    temporal_axis
from relation import Relation
from fact import Fact
from benchmark import CounterFactualExample, RecentlyAddedExample, Dataset
from queryexecutor import QueryExecutor


def all_relevant_facts_given_list_of_subjects(subjects: list, limit: int = None):
    facts = []
    for i, subject_id in enumerate(subjects):
        if (i+1) % 100 == 0:
            print(f'{i+1}/{len(subjects)}')
        relevant_relation_ids = ent_to_relation_ids(subject_id)
        for relation_id in relevant_relation_ids:
            relation_enum = Relation.id_to_enum(relation_id)
            if relation_enum is None:
                continue
            targets = subject_relation_to_targets(subject_id, relation_id)
            for target_id in targets:
                facts.append((subject_id, relation_enum, target_id))
        if limit is not None and len(facts) >= limit:
            break
    return facts

def construct_fake_edits_benchmark(facts: list):
    dataset_list = []
    cnt = 0
    for subject_id, relation, target_id in facts:
        relation2optional_targets = load_json('./wikidata/relation2optional_targets_new.json')
        relation_formal_name = relation.formal_name()
        if relation_formal_name not in relation2optional_targets:
            continue
        optional_targets = relation2optional_targets[relation.formal_name()]
        random_target_id = ent_label2id(random.sample(optional_targets, 1)[0])
        if random_target_id is None:
            continue
        dataset_list.append(build_fake_dataset_example(subject_id, relation, random_target_id, target_id))
        cnt += 1
        if cnt % 100 == 0:
            print(f'{cnt}/{len(facts)}')
    return Dataset(dataset_list)

def construct_two_hop_dataset(path: str, limit: int, facts_limit: int = None,
                                                   limit_subjects: int = None, limit_num_of_facts: int = None):
    subjects_json = load_json(path)
    subject_list = []
    for bucket in subjects_json:
        subject_list.extend(bucket)
    subject_ids = [ent_label2id(subject_label) for subject_label in subject_list]
    if limit_subjects is not None:
        subject_ids = random.sample(subject_ids, min(limit_subjects, len(subject_ids)))
    print('extracting facts..')
    if limit_num_of_facts is None:
        all_relevant_facts = all_relevant_facts_given_list_of_subjects(subject_ids, facts_limit)
    else:
        all_relevant_facts = sample_relevant_facts_given_list_of_subjects(subject_ids, limit_num_of_facts, facts_limit)
    print(f'have got {len(all_relevant_facts)} relevant facts to sample from')
    print('building dataset..')
    random.shuffle(all_relevant_facts)
    all_relevant_facts = random.sample(all_relevant_facts, min(limit, len(all_relevant_facts)))
    dataset = construct_fake_edits_benchmark(all_relevant_facts)

IndentationError: expected an indented block (1038806190.py, line 1)

In [46]:
from relation import Relation
from two_hop_phrases import relation_couple_to_phrase
len(Relation)
relation_two_hops = []
for rel1 in Relation:
    for rel2 in Relation:
        if relation_couple_to_phrase(rel1, rel2) is not None:
            relation_two_hops.append((rel1, rel2))
len(relation_two_hops)
            

532